In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Prepared data

In [3]:
bc = datasets.load_breast_cancer()

In [4]:
X = bc.data.astype(np.float32)
y = bc.target.astype(np.float32)

In [5]:
n_sample,n_features = X.shape[0], X.shape[1]
n_sample,n_features 

(569, 30)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1234)

In [7]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
X_train = torch.from_numpy(X_train).cuda()
y_train = torch.from_numpy(y_train).view(y_train.shape[0],1).cuda()
X_test = torch.from_numpy(X_test).cuda()
y_test = torch.from_numpy(y_test).view(y_test.shape[0],1).cuda()

# Constants

In [9]:
input_size = n_features
output_size = 1
learning_rate = 0.01
n_iters = 100

# Model

In [10]:
class LogisticRegression(nn.Module):
    def __init__(self,input_dim,output_dim):
        super(LogisticRegression,self).__init__()
        
        self.LR = nn.Sequential(
                                nn.Linear(input_dim,output_dim),
                                nn.Sigmoid()
                                    )
    def forward(self,x):
        return self.LR(x)

In [11]:
model = LogisticRegression(input_size,output_size)

# Loss & optimizer

In [12]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

# Training loop

In [13]:
for epoch in range(n_iters):
    
    # Predict
    y_pred = model(X_train)
    # Compare
    loss = criterion(y_pred,y_train)
    # Learn
    dw = loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (epoch+1) % 10 ==0:
        print(f'epoch {epoch+1}: loss = {loss.item():.4f}')
    

epoch 10: loss = 0.3557
epoch 20: loss = 0.2083
epoch 30: loss = 0.1514
epoch 40: loss = 0.1248
epoch 50: loss = 0.1093
epoch 60: loss = 0.0990
epoch 70: loss = 0.0916
epoch 80: loss = 0.0858
epoch 90: loss = 0.0811
epoch 100: loss = 0.0772


In [14]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum()/float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9561
